<a href="https://colab.research.google.com/github/ADionysopoulos/efood_assesment/blob/main/BigQuery_bquxjob_4268ce04_18b9ad7f5ea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
import numpy as np
import pandas as pd
from itertools import product

project = 'efood2023-404109' # Project ID inserted based on the query results selected to explore
location = 'EU' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=efood2023-404109:EU:bquxjob_4268ce04_18b9ad7f5ea)
back to BigQuery to edit the query within the BigQuery user interface.

In [3]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_1b334270_18ba011cc46') # Job ID inserted based on the query results selected to explore
print(job.query)


-- SELECT 
--   user_id AS `Users`,
--   count(order_id) As count_order_id,
--   sum(amount) As sum_spending_amount,
--   sum(amount) / count(order_id) As mean_spending_amount,
--   count(NULLIF(coupon_discount_amount, 0)) As count_coupon_discount_amount,
--   sum(coupon_discount_amount) As sum_coupon_discount_amount,
--   CASE WHEN
--    count(NULLIF(coupon_discount_amount, 0)) <> 0 
--       THEN sum(coupon_discount_amount) / count(NULLIF(coupon_discount_amount, 0))
--     ELSE NULL
--   END AS mean_spending_amount
--   FROM `efood2023-404109.main_assessment.orders` 
--   group by user_id
--   order by count_order_id DESC

SELECT * FROM `efood2023-404109.main_assessment.orders`
  WHERE cuisine = "Breakfast"


# Analyzing Coupon Effects on Customer Behavior no grouping (general effect)  

The following  code snippet is part of a data analysis process and focuses on evaluating the impact of coupons on customer behavior. The code performs various steps, including data filtering, calculating coupon effects, and analyzing coupon event patterns. It uses data manipulation techniques, such as pivot tables and data structures, to compare user behavior before and after coupon events, and it calculates differences in various metrics. This code helps gain insights into how coupon campaigns influence customer behavior, allowing for data-driven decision-making in marketing strategies.

In [4]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_1b334270_18ba011cc46') # Job ID inserted based on the query results selected to explore
results_init = job.to_dataframe()

In [6]:
# Keep only the users who had took coupon in the past
results = results_init
results_init['order_timestamp_day'] = results_init['order_timestamp'].dt.day_of_year
results_init['coupon_discount'] = results_init['coupon_discount_amount'] > 0
users_with_coupon = list(results_init.loc[results_init['coupon_discount'].values, "user_id"].unique())
results = results.set_index('user_id')
target_coupon_results = results.loc[users_with_coupon,:]
target_coupon_results = target_coupon_results.reset_index()

In [7]:
coupon_effect = pd.pivot_table(target_coupon_results,
                               index = ["user_id", "order_timestamp_day"],
                               values = ["order_id", "amount", "coupon_discount_amount"],
                               aggfunc= {"order_id": "count", "amount": [np.median, np.sum], "coupon_discount_amount": np.sum})
coupon_effect.columns = ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']

In [8]:
user_ids = coupon_effect.reset_index()['user_id'].unique()
days_range = range(results_init['order_timestamp_day'].unique().min(), results_init['order_timestamp_day'].unique().max() + 1)

df_index_user_id = [user_id for user_id in user_ids for _ in days_range]
df_index_day     = [day for _ in user_ids for day in days_range]

In [9]:
extended_coupon_effect = pd.DataFrame(columns = coupon_effect.columns)
extended_coupon_effect["user_id"] = df_index_user_id
extended_coupon_effect["order_timestamp_day"] = df_index_day
extended_coupon_effect = extended_coupon_effect.set_index(["user_id", "order_timestamp_day"])

In [10]:
extended_coupon_effect.loc[coupon_effect.index, ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']] = coupon_effect[['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']]
extended_coupon_effect = extended_coupon_effect.fillna(0)
extended_coupon_effect["coupon event"] = extended_coupon_effect['coupon_discount_amount_sum'] > 0

In [11]:
A = extended_coupon_effect.reset_index()
events = A.loc[A["coupon event"], ["user_id", "order_timestamp_day","coupon event"]]

coupon_case_bef_df = pd.DataFrame(columns = extended_coupon_effect.columns)
coupon_case_after_df = pd.DataFrame(columns = extended_coupon_effect.columns)

coupon_case = 0
coupon_window = 3

extended_coupon_effect["Aggr_coupon_team_bef"] = np.nan
extended_coupon_effect["Aggr_coupon_team_aft"] = np.nan

for index, row in events.iterrows():

  user_id = row["user_id"]
  coupon_day = row["order_timestamp_day"]

  if (coupon_day - coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1)) & ((coupon_day + coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1))):

    index_prev= []

    for k in range(coupon_day - coupon_window, coupon_day):
      index_prev.append([user_id, k])

    index_after= []

    for k in range(coupon_day + 1, coupon_day + coupon_window + 1):
      index_after.append([user_id, k])

    if (extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"].fillna(0).sum() == 0) & (extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"].fillna(0).sum() == 0):
      extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"] = coupon_case
      extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"] = coupon_case

    coupon_case +=1



In [12]:
A1 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_bef", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})
A2 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_aft", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})

In [13]:
A1.columns
diffs = A2 - A1
diffs.describe()

,amount_median,order_id_count
count,6371.000000,6371.000000
mean,-0.020212,-0.048815
std,2.318656,1.545227
min,-16.366667,-8.000000
25%,-1.066667,-1.000000
50%,0.000000,0.000000
75%,0.966667,1.000000
max,21.333333,15.000000


# Analyzing Coupon Effects on Customer Behavior by User Class


In [19]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_1b334270_18ba011cc46') # Job ID inserted based on the query results selected to explore
results_init = job.to_dataframe()

for i in results_init['user_class_name'].unique():
  # Copy the initial results
  results_init = job.to_dataframe()

  # Filter results for the current user_class_name
  results_init = results_init[results_init['user_class_name'] == i ]

  # Create a copy of results for further analysis
  results = results_init

  # Keep only the users who had took coupon in the past
  results_init['order_timestamp_day'] = results_init['order_timestamp'].dt.day_of_year
  results_init['coupon_discount'] = results_init['coupon_discount_amount'] > 0
  users_with_coupon = list(results_init.loc[results_init['coupon_discount'].values, "user_id"].unique())
  results = results.set_index('user_id')
  target_coupon_results = results.loc[users_with_coupon,:]
  target_coupon_results = target_coupon_results.reset_index()

  # Create a pivot table for coupon effect analysis
  coupon_effect = pd.pivot_table(target_coupon_results,
                                index = ["user_id", "order_timestamp_day"],
                                values = ["order_id", "amount", "coupon_discount_amount"],
                                aggfunc= {"order_id": "count", "amount": [np.median, np.sum],"coupon_discount_amount": np.sum})
  coupon_effect.columns = ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']

  # Prepare data structures for analysis
  user_ids = coupon_effect.reset_index()['user_id'].unique()
  days_range = range(results_init['order_timestamp_day'].unique().min(), results_init['order_timestamp_day'].unique().max() + 1)

  df_index_user_id = [user_id for user_id in user_ids for _ in days_range]
  df_index_day = [day for _ in user_ids for day in days_range]

  extended_coupon_effect =pd.DataFrame(columns = coupon_effect.columns)
  extended_coupon_effect["user_id"] = df_index_user_id
  extended_coupon_effect["order_timestamp_day"] = df_index_day
  extended_coupon_effect = extended_coupon_effect.set_index(["user_id", "order_timestamp_day"])

  extended_coupon_effect.loc[coupon_effect.index, ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']] = coupon_effect[['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']]
  extended_coupon_effect = extended_coupon_effect.fillna(0)
  extended_coupon_effect["coupon event"] = extended_coupon_effect['coupon_discount_amount_sum'] > 0

  # Create data structures for coupon event analysis
  A = extended_coupon_effect.reset_index()
  events = A.loc[A["coupon event"], ["user_id", "order_timestamp_day","coupon event"]]

  coupon_case_bef_df = pd.DataFrame(columns = extended_coupon_effect.columns)
  coupon_case_after_df = pd.DataFrame(columns = extended_coupon_effect.columns)

  coupon_case = 0
  coupon_window = 3

  extended_coupon_effect["Aggr_coupon_team_bef"] = np.nan
  extended_coupon_effect["Aggr_coupon_team_aft"] = np.nan

  for index, row in events.iterrows():

    user_id = row["user_id"]
    coupon_day = row["order_timestamp_day"]

    if (coupon_day - coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1)) & ((coupon_day + coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1))):

      index_prev= []

      for k in range(coupon_day - coupon_window, coupon_day):
        index_prev.append([user_id, k])

      index_after= []

      for k in range(coupon_day + 1, coupon_day + coupon_window + 1):
        index_after.append([user_id, k])

      if (extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"].fillna(0).sum() == 0) & (extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"].fillna(0).sum() == 0):
        extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"] = coupon_case
        extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"] = coupon_case

      coupon_case +=1
  # Create pivot tables and calculate differences
  A1 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_bef", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})
  A2 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_aft", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})


  diffs = A2 - A1
  print(i)

  print(diffs.describe())
  print("----------------------------------------------------------------")


Loyal
       amount_median  order_id_count
count    1915.000000     1915.000000
mean       -0.037681       -0.084595
std         1.678881        1.450740
min        -7.566667       -6.000000
25%        -0.983333       -1.000000
50%         0.000000        0.000000
75%         0.833333        1.000000
max         8.766667        6.000000
----------------------------------------------------------------
All Star
       amount_median  order_id_count
count    3308.000000     3308.000000
mean       -0.085144       -0.071644
std         2.816450        1.815501
min       -16.366667       -8.000000
25%        -1.533333       -1.000000
50%         0.000000        0.000000
75%         1.366667        1.000000
max        21.333333       15.000000
----------------------------------------------------------------
Infrequent
       amount_median  order_id_count
count     266.000000      266.000000
mean        0.265602        0.112782
std         1.213527        0.462220
min        -4.500000       -1.

<ipython-input-19-73e621c00139>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_init['order_timestamp_day'] = results_init['order_timestamp'].dt.day_of_year
<ipython-input-19-73e621c00139>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_init['coupon_discount'] = results_init['coupon_discount_amount'] > 0


High Spenders
       amount_median  order_id_count
count     187.000000      187.000000
mean        0.088271        0.021390
std         2.486215        0.796209
min        -7.733333       -2.000000
25%         0.000000        0.000000
50%         0.000000        0.000000
75%         0.533333        0.000000
max         9.700000        3.000000
----------------------------------------------------------------


# Analyzing Coupon Effects on Customer Behavior by User city



In [15]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_1b334270_18ba011cc46') # Job ID inserted based on the query results selected to explore
results_init = job.to_dataframe()
mask = (results_init["user_class_name"] == "Infrequent") | (results_init["user_class_name"] == "Common User") | (results_init["user_class_name"] == "High Spenders")
results_init = results_init[mask]

for i in results_init['city'].unique():

  results_init = job.to_dataframe()
  mask = (results_init["user_class_name"] == "Infrequent") | (results_init["user_class_name"] == "Common User") | (results_init["user_class_name"] == "High Spenders")
  results_init = results_init[mask]
  results_init = results_init[results_init['city'] == i ]

  results = results_init
  # Keep only the users who had took coupon in the past
  results_init['order_timestamp_day'] = results_init['order_timestamp'].dt.day_of_year
  results_init['coupon_discount'] = results_init['coupon_discount_amount'] > 0
  users_with_coupon = list(results_init.loc[results_init['coupon_discount'].values, "user_id"].unique())
  results = results.set_index('user_id')
  target_coupon_results = results.loc[users_with_coupon,:]
  target_coupon_results = target_coupon_results.reset_index()

  coupon_effect = pd.pivot_table(target_coupon_results,
                                index = ["user_id", "order_timestamp_day"],
                                values = ["order_id", "amount", "coupon_discount_amount"],
                                aggfunc= {"order_id": "count", "amount": [np.median, np.sum],"coupon_discount_amount": np.sum})
  coupon_effect.columns = ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']

  user_ids = coupon_effect.reset_index()['user_id'].unique()
  days_range = range(results_init['order_timestamp_day'].unique().min(), results_init['order_timestamp_day'].unique().max() + 1)

  df_index_user_id = [user_id for user_id in user_ids for _ in days_range]
  df_index_day = [day for _ in user_ids for day in days_range]

  extended_coupon_effect =pd.DataFrame(columns = coupon_effect.columns)
  extended_coupon_effect["user_id"] = df_index_user_id
  extended_coupon_effect["order_timestamp_day"] = df_index_day
  extended_coupon_effect = extended_coupon_effect.set_index(["user_id", "order_timestamp_day"])

  extended_coupon_effect.loc[coupon_effect.index, ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']] = coupon_effect[['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']]
  extended_coupon_effect = extended_coupon_effect.fillna(0)
  extended_coupon_effect["coupon event"] = extended_coupon_effect['coupon_discount_amount_sum'] > 0

  A = extended_coupon_effect.reset_index()
  events = A.loc[A["coupon event"], ["user_id", "order_timestamp_day","coupon event"]]

  coupon_case_bef_df = pd.DataFrame(columns = extended_coupon_effect.columns)
  coupon_case_after_df = pd.DataFrame(columns = extended_coupon_effect.columns)

  coupon_case = 0
  coupon_window = 3

  extended_coupon_effect["Aggr_coupon_team_bef"] = np.nan
  extended_coupon_effect["Aggr_coupon_team_aft"] = np.nan

  for index, row in events.iterrows():

    user_id = row["user_id"]
    coupon_day = row["order_timestamp_day"]

    if (coupon_day - coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1)) & ((coupon_day + coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1))):

      index_prev= []

      for k in range(coupon_day - coupon_window, coupon_day):
        index_prev.append([user_id, k])

      index_after= []

      for k in range(coupon_day + 1, coupon_day + coupon_window + 1):
        index_after.append([user_id, k])

      if (extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"].fillna(0).sum() == 0) & (extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"].fillna(0).sum() == 0):
        extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"] = coupon_case
        extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"] = coupon_case

      coupon_case +=1

  A1 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_bef", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})
  A2 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_aft", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})

  A1.columns
  diffs = A2 - A1
  print(i)
  print(diffs.describe())
  print("----------------------------------------------------------------")

Άρτα
       amount_median  order_id_count
count      32.000000       32.000000
mean       -0.019792        0.000000
std         0.696219        0.508001
min        -2.133333       -1.000000
25%         0.000000        0.000000
50%         0.000000        0.000000
75%         0.000000        0.000000
max         1.500000        1.000000
----------------------------------------------------------------
Αίγιο
       amount_median  order_id_count
count      26.000000       26.000000
mean       -0.075641        0.076923
std         1.069852        0.627572
min        -4.600000       -1.000000
25%         0.000000        0.000000
50%         0.000000        0.000000
75%         0.000000        0.000000
max         1.666667        1.000000
----------------------------------------------------------------
Δράμα
       amount_median  order_id_count
count      68.000000       68.000000
mean        0.036275       -0.029412
std         1.938845        0.863360
min        -5.466667       -4.000000
25

# Analyzing Coupon Effects on Customer Behavior by Device

In [16]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_1b334270_18ba011cc46') # Job ID inserted based on the query results selected to explore
results_init = job.to_dataframe()
mask = (results_init["user_class_name"] == "Infrequent") | (results_init["user_class_name"] == "Common User") | (results_init["user_class_name"] == "High Spenders")
results_init = results_init[mask]
mask2 = (results_init["device"] != "Android")
results_init.loc[mask2, "device"] = "Not Android"

for i in results_init['device'].unique():

  results_init = job.to_dataframe()
  mask = (results_init["user_class_name"] == "Infrequent") | (results_init["user_class_name"] == "Common User") | (results_init["user_class_name"] == "High Spenders")
  results_init = results_init[mask]
  mask2 = (results_init["device"] != "Android")
  results_init.loc[mask2, "device"] = "Not Android"
  results_init = results_init[results_init['device'] == i ]

  results = results_init
  # Keep only the users who had took coupon in the past
  results_init['order_timestamp_day'] = results_init['order_timestamp'].dt.day_of_year
  results_init['coupon_discount'] = results_init['coupon_discount_amount'] > 0
  users_with_coupon = list(results_init.loc[results_init['coupon_discount'].values, "user_id"].unique())
  results = results.set_index('user_id')
  target_coupon_results = results.loc[users_with_coupon,:]
  target_coupon_results = target_coupon_results.reset_index()

  coupon_effect = pd.pivot_table(target_coupon_results,
                                index = ["user_id", "order_timestamp_day"],
                                values = ["order_id", "amount", "coupon_discount_amount"],
                                aggfunc= {"order_id": "count", "amount": [np.median, np.sum],"coupon_discount_amount": np.sum})
  coupon_effect.columns = ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']

  user_ids = coupon_effect.reset_index()['user_id'].unique()
  days_range = range(results_init['order_timestamp_day'].unique().min(), results_init['order_timestamp_day'].unique().max() + 1)

  df_index_user_id = [user_id for user_id in user_ids for _ in days_range]
  df_index_day = [day for _ in user_ids for day in days_range]

  extended_coupon_effect =pd.DataFrame(columns = coupon_effect.columns)
  extended_coupon_effect["user_id"] = df_index_user_id
  extended_coupon_effect["order_timestamp_day"] = df_index_day
  extended_coupon_effect = extended_coupon_effect.set_index(["user_id", "order_timestamp_day"])

  extended_coupon_effect.loc[coupon_effect.index, ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']] = coupon_effect[['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']]
  extended_coupon_effect = extended_coupon_effect.fillna(0)
  extended_coupon_effect["coupon event"] = extended_coupon_effect['coupon_discount_amount_sum'] > 0

  A = extended_coupon_effect.reset_index()
  events = A.loc[A["coupon event"], ["user_id", "order_timestamp_day","coupon event"]]

  coupon_case_bef_df = pd.DataFrame(columns = extended_coupon_effect.columns)
  coupon_case_after_df = pd.DataFrame(columns = extended_coupon_effect.columns)

  coupon_case = 0
  coupon_window = 3

  extended_coupon_effect["Aggr_coupon_team_bef"] = np.nan
  extended_coupon_effect["Aggr_coupon_team_aft"] = np.nan

  for index, row in events.iterrows():

    user_id = row["user_id"]
    coupon_day = row["order_timestamp_day"]

    if (coupon_day - coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1)) & ((coupon_day + coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1))):

      index_prev= []

      for k in range(coupon_day - coupon_window, coupon_day):
        index_prev.append([user_id, k])

      index_after= []

      for k in range(coupon_day + 1, coupon_day + coupon_window + 1):
        index_after.append([user_id, k])

      if (extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"].fillna(0).sum() == 0) & (extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"].fillna(0).sum() == 0):
        extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"] = coupon_case
        extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"] = coupon_case

      coupon_case +=1

  A1 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_bef", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})
  A2 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_aft", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})

  A1.columns
  diffs = A2 - A1
  print(i)
  print(diffs.describe())
  print("----------------------------------------------------------------")

Not Android
       amount_median  order_id_count
count     384.000000      384.000000
mean       -0.000851       -0.031250
std         1.867459        0.753798
min        -7.133333       -4.000000
25%         0.000000        0.000000
50%         0.000000        0.000000
75%         0.000000        0.000000
max         8.933333        2.000000
----------------------------------------------------------------
Android
       amount_median  order_id_count
count     448.000000      448.000000
mean        0.378311        0.131696
std         1.791135        0.647193
min        -7.733333       -3.000000
25%         0.000000        0.000000
50%         0.000000        0.000000
75%         0.000000        0.000000
max        13.033333        3.000000
----------------------------------------------------------------


# Analyzing Coupon Effects on Customer Behavior by Paid cash

In [17]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_1b334270_18ba011cc46') # Job ID inserted based on the query results selected to explore
results_init = job.to_dataframe()
mask = (results_init["user_class_name"] == "Infrequent") | (results_init["user_class_name"] == "Common User") | (results_init["user_class_name"] == "High Spenders")
results_init = results_init[mask]

for i in results_init['paid_cash'].unique():

  results_init = job.to_dataframe()
  mask = (results_init["user_class_name"] == "Infrequent") | (results_init["user_class_name"] == "Common User") | (results_init["user_class_name"] == "High Spenders")
  results_init = results_init[mask]
  results_init = results_init[results_init['paid_cash'] == i]

  results = results_init
  # Keep only the users who had took coupon in the past
  results_init['order_timestamp_day'] = results_init['order_timestamp'].dt.day_of_year
  results_init['coupon_discount'] = results_init['coupon_discount_amount'] > 0
  users_with_coupon = list(results_init.loc[results_init['coupon_discount'].values, "user_id"].unique())
  results = results.set_index('user_id')
  target_coupon_results = results.loc[users_with_coupon,:]
  target_coupon_results = target_coupon_results.reset_index()

  coupon_effect = pd.pivot_table(target_coupon_results,
                                index = ["user_id", "order_timestamp_day"],
                                values = ["order_id", "amount", "coupon_discount_amount"],
                                aggfunc= {"order_id": "count", "amount": [np.median, np.sum],"coupon_discount_amount": np.sum})
  coupon_effect.columns = ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']

  user_ids = coupon_effect.reset_index()['user_id'].unique()
  days_range = range(results_init['order_timestamp_day'].unique().min(), results_init['order_timestamp_day'].unique().max() + 1)

  df_index_user_id = [user_id for user_id in user_ids for _ in days_range]
  df_index_day = [day for _ in user_ids for day in days_range]

  extended_coupon_effect =pd.DataFrame(columns = coupon_effect.columns)
  extended_coupon_effect["user_id"] = df_index_user_id
  extended_coupon_effect["order_timestamp_day"] = df_index_day
  extended_coupon_effect = extended_coupon_effect.set_index(["user_id", "order_timestamp_day"])

  extended_coupon_effect.loc[coupon_effect.index, ['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']] = coupon_effect[['amount_median', 'amount_sum', 'coupon_discount_amount_sum', 'order_id_count']]
  extended_coupon_effect = extended_coupon_effect.fillna(0)
  extended_coupon_effect["coupon event"] = extended_coupon_effect['coupon_discount_amount_sum'] > 0

  A = extended_coupon_effect.reset_index()
  events = A.loc[A["coupon event"], ["user_id", "order_timestamp_day","coupon event"]]

  coupon_case_bef_df = pd.DataFrame(columns = extended_coupon_effect.columns)
  coupon_case_after_df = pd.DataFrame(columns = extended_coupon_effect.columns)

  coupon_case = 0
  coupon_window = 3

  extended_coupon_effect["Aggr_coupon_team_bef"] = np.nan
  extended_coupon_effect["Aggr_coupon_team_aft"] = np.nan

  for index, row in events.iterrows():

    user_id = row["user_id"]
    coupon_day = row["order_timestamp_day"]

    if (coupon_day - coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1)) & ((coupon_day + coupon_window  in range(results_init["order_timestamp_day"].unique().min(), results_init["order_timestamp_day"].unique().max()+1))):

      index_prev= []

      for k in range(coupon_day - coupon_window, coupon_day):
        index_prev.append([user_id, k])

      index_after= []

      for k in range(coupon_day + 1, coupon_day + coupon_window + 1):
        index_after.append([user_id, k])

      if (extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"].fillna(0).sum() == 0) & (extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"].fillna(0).sum() == 0):
        extended_coupon_effect.loc[index_prev, "Aggr_coupon_team_bef"] = coupon_case
        extended_coupon_effect.loc[index_after, "Aggr_coupon_team_aft"] = coupon_case

      coupon_case +=1

  A1 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_bef", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})
  A2 = pd.pivot_table(extended_coupon_effect, index = "Aggr_coupon_team_aft", values= ['amount_median', "order_id_count"], aggfunc = {"amount_median":np.mean, "order_id_count": "sum"})

  A1.columns
  diffs = A2 - A1
  print(i)
  print(diffs.describe())
  print("----------------------------------------------------------------")

False
       amount_median  order_id_count
count     696.000000      696.000000
mean        0.155924        0.056034
std         1.568797        0.558537
min       -10.300000       -2.000000
25%         0.000000        0.000000
50%         0.000000        0.000000
75%         0.000000        0.000000
max        13.033333        3.000000
----------------------------------------------------------------
True
       amount_median  order_id_count
count     136.000000      136.000000
mean        0.204412        0.029412
std         2.164332        0.934453
min        -7.100000       -4.000000
25%         0.000000        0.000000
50%         0.000000        0.000000
75%         0.141667        0.000000
max         7.516667        2.000000
----------------------------------------------------------------
